## Extracting the Coordinates

In [1]:
import requests
import json

In [2]:
# Overpass API endpoint
url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"

In [3]:
# Example query: All cycling routes in Europe (bicycle routes)
#-----------------------------------------------CHANGE COORDS HERE--------------------------------------#
query = """
[out:json][timeout:900];
(
  relation["route"="bicycle"](42.98, 4.2283, 45.16, 7.7189);
);
out geom;
"""
#-----------------------------------------------CHANGE COORDS HERE--------------------------------------#

In [4]:
# Make the request
response = requests.get(url, params={'data': query})

In [7]:
# Check success
if response.status_code == 200:
    data = response.json()
    print("✅ Success:", len(data['elements']), "routes found")
else:
    print("❌ Error:", response.status_code)

✅ Success: 525 routes found


In [8]:
# Save response to file
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
with open("Turbo_coords_provence.json", "w") as f:
    json.dump(data, f)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [9]:
import pandas as pd

# Create empty list to store results
routes_summary = []

# Loop through each route in the dataset
for route in data['elements']:
    if route['type'] != 'relation':
        continue  # skip non-route elements

    coords = []
    for member in route.get('members', []):
        if 'geometry' in member:
            segment_coords = [(pt['lon'], pt['lat']) for pt in member['geometry']]
            coords.extend(segment_coords)

    if coords:
        start_coord = coords[0]
        end_coord = coords[-1]
        routes_summary.append({
            'id': route.get('id'),
            'name': route.get('tags', {}).get('name', 'Unnamed route'),
            'start_lon': start_coord[0],
            'start_lat': start_coord[1],
            'end_lon': end_coord[0],
            'end_lat': end_coord[1],
            'num_points': len(coords)
        })

In [10]:
# Convert results to DataFrame
routes_df = pd.DataFrame(routes_summary)

In [11]:
routes_df.to_csv("turbo_start_end_coords_provence.csv", index=False)

In [16]:
routes_df

,id,name,start_lon,start_lat,end_lon,end_lat,num_points
0,168466,Boucles du 13 Camargue,4.670930,43.514253,4.702068,43.464042,533
1,168472,Boucles du 13 - La Chaîne des Côtes,5.393765,43.685059,5.394779,43.687317,857
2,168474,Boucles du 13 - Saintes Maries,4.429896,43.499538,4.429896,43.499538,646
3,168475,Les boucles du 13 - Autour de Silvacane,5.186004,43.731102,5.241505,43.728157,1175
4,168476,D’Arles à l’étang du Vaccarès,4.627061,43.675217,4.626507,43.675139,1388
...,...,...,...,...,...,...,...
520,19816737,"Vallées, châtaignes et volcans",4.358189,44.717815,4.386783,44.717057,1253
521,19824619,Uzès à Saint-Quentin-la-Poterie,4.414556,44.027059,4.438144,44.042723,150
522,19849376,V65,5.251022,43.368071,5.251022,43.368071,31
523,19849377,V65,5.092375,43.333177,5.103856,43.329831,115


# Extracting the features UK First 2,000

In [12]:
import os

In [13]:
AK = os.environ["AK"]

In [14]:
import openrouteservice
import pandas as pd
import json
import time
import os

In [15]:
# Initialize client
client = openrouteservice.Client(key=AK)  # Replace AK with your ORS API key

In [15]:
results = []
save_interval = 10

In [20]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
save_path = "ors_Features_Provence.json"
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [21]:
test_df = routes_df.iloc[0:1]

In [22]:
# Folder to save raw API responses
raw_dir = "raw_ors_responses_Provence_test"
os.makedirs(raw_dir, exist_ok=True)

for i, row in test_df.iterrows():
    start = [row['start_lon'], row['start_lat']]
    end = [row['end_lon'], row['end_lat']]

    try:
        # --- Call ORS API ---
        route = client.directions(
            coordinates=[start, end],
            profile='cycling-regular',
            format='geojson',
            elevation=True,
            extra_info=['surface', 'waytype', 'waycategory', 'steepness']
        )

        # --- Save JSON response ---
        raw_filename = f"{raw_dir}/route_{row['id']}.json"
        with open(raw_filename, "w") as f:
            json.dump(route, f, indent=2)

        print(f"✅ Saved route {row['id']} ({i+1}/{len(test_df)})")
        time.sleep(1.5)  # rate limit protection

    except Exception as e:
        print(f"❌ Error on route {row['id']}: {e}")
        continue


✅ Saved route 168466 (1/1)


In [24]:
import json
import os
import pandas as pd

raw_dir = "raw_ors_responses_Provence_test"
results = []

# Loop through all JSON files
for filename in os.listdir(raw_dir):
    if not filename.endswith(".json"):
        continue

    filepath = os.path.join(raw_dir, filename)

    with open(filepath, "r") as f:
        route = json.load(f)

    try:
        props = route['features'][0]['properties']
        summary = props['summary']
        extras = props.get('extras', {})
        segments = props.get('segments', [{}])
        steps = segments[0].get('steps', [])

        # --- Extract features ---
        route_id = filename.replace("route_", "").replace(".json", "")
        distance = summary.get('distance')
        duration = summary.get('duration')
        ascent = props.get('ascent')
        descent = props.get('descent')
        turns = len([s for s in steps if s.get('type') in range(8)])
        num_steps = len(steps)

        results.append({
            'id': route_id,
            'distance_m': distance,
            'duration_s': duration,
            'ascent_m': ascent,
            'descent_m': descent,
            'turns': turns,
            'steps': num_steps,
            'surface': extras.get('surface', {}).get('values', []),
            'waytype': extras.get('waytype', {}).get('values', []),
            'waycategory': extras.get('waycategory', {}).get('values', []),
            'steepness': extras.get('steepness', {}).get('values', []),
        })

        print(f"✅ Parsed {filename}")

    except Exception as e:
        print(f"⚠️ Error parsing {filename}: {e}")

# --- Save to CSV ---
df = pd.DataFrame(results)
df.to_csv("processed_routes_Provence_test.csv", index=False)
print(f"✅ Saved {len(df)} processed routes to CSV")


✅ Parsed route_168466.json
✅ Saved 1 processed routes to CSV


In [19]:
for i, row in test_df.iterrows():
    start = (row['start_lon'], row['start_lat'])
    end = (row['end_lon'], row['end_lat'])

    try:
        route = client.directions(
            coordinates=[start, end],
            profile='cycling-regular',
            format='geojson',
            elevation=True,
            instructions=True,
            extra_info=['surface', 'waytype', 'waycategory', 'steepness'],
        )



        # --- SAVE RAW ORS RESPONSE FOR DEBUGGING ---
        #-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
        raw_dir = "raw_ors_responses_Provence"
        #-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
        os.makedirs(raw_dir, exist_ok=True)

        raw_filename = f"{raw_dir}/route_{row['id']}.json"

        with open(raw_filename, "w") as raw_file:
            json.dump(route, raw_file, indent=2)
        # -------------------------------------------


        props = route['features'][0]['properties']
        summary = props['summary']
        extras = props.get('extras', {})
        segments = props['segments']
        steps = segments[0]['steps']
        turn_steps = [s for s in steps if s['type'] in {0,1,2,3,4,5,6,7}]
        turns = len(turn_steps)
        steps = len(steps)


        results.append({
            'id': row['id'],
            'name': row['name'],
            'distance_m': summary.get('distance'),
            'duration_s': summary.get('duration'),
            'ascent_m': [props['ascent']],
            'descent_m': [props['descent']],
            'steps': steps,
            'turns': turns,
            'surface': extras.get('surface', {}).get('values', []),
            'waytype': extras.get('waytype', {}).get('values', []),
            'waycategory': extras.get('waycategory', {}).get('values', []),
            'steepness': extras.get('steepness', {}).get('values', []),
        })

        print(f"✅ Route {i+1} processed successfully")
        time.sleep(1.5)  # rate-limit protection

        # Save partial file
        if (i + 1) % save_interval == 0:
            with open(save_path, "w") as f:
                json.dump(results, f, indent=2)
            print(f"💾 Saved partial results after {i+1} routes")

    except Exception as e:
        print(f"❌ Error on route {row['id']}: {e}")
        continue

✅ Route 1 processed successfully
✅ Route 2 processed successfully
✅ Route 3 processed successfully
✅ Route 4 processed successfully
✅ Route 5 processed successfully
✅ Route 6 processed successfully
✅ Route 7 processed successfully
✅ Route 8 processed successfully
✅ Route 9 processed successfully
✅ Route 10 processed successfully
💾 Saved partial results after 10 routes
✅ Route 11 processed successfully
✅ Route 12 processed successfully
✅ Route 13 processed successfully
✅ Route 14 processed successfully
✅ Route 15 processed successfully
✅ Route 16 processed successfully
✅ Route 17 processed successfully
✅ Route 18 processed successfully
✅ Route 19 processed successfully
✅ Route 20 processed successfully
💾 Saved partial results after 20 routes
✅ Route 21 processed successfully
✅ Route 22 processed successfully
✅ Route 23 processed successfully
✅ Route 24 processed successfully
✅ Route 25 processed successfully
✅ Route 26 processed successfully
✅ Route 27 processed successfully
✅ Route 28 

In [31]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
final_path = "ors_route_results_Provence.json"
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [32]:
final_path

'ors_route_results_Provence.json'

In [33]:
with open(final_path, "w") as f:
    json.dump(results, f, indent=2)

In [34]:
print(f"\n🎉 Finished run — processed {len(results)} routes.")
print(f"📁 Full output saved to {final_path}")


🎉 Finished run — processed 523 routes.
📁 Full output saved to ors_route_results_Provence.json


In [35]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
with open("ors_route_results_Provence.json", "r") as f:
    results = json.load(f)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [36]:
# Convert to a DataFrame
df = pd.DataFrame(results)

In [37]:
# Display the first few rows
display(df.head(1))

,id,name,distance_m,duration_s,ascent_m,descent_m,steps,turns,surface,waytype,waycategory,steepness
0,168466,Boucles du 13 Camargue,15578.0,3116.4,[30.3],[30.3],6,2,"[[0, 129, 3], [129, 131, 0], [131, 185, 3]]","[[0, 185, 2]]","[[0, 185, 0]]","[[0, 185, 0]]"


In [38]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
df.to_csv("Provence.csv", index=False)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [39]:
df

,id,name,distance_m,duration_s,ascent_m,descent_m,steps,turns,surface,waytype,waycategory,steepness
0,168466,Boucles du 13 Camargue,15578.0,3116.4,[30.3],[30.3],6,2,"[[0, 129, 3], [129, 131, 0], [131, 185, 3]]","[[0, 185, 2]]","[[0, 185, 0]]","[[0, 185, 0]]"
1,168472,Boucles du 13 - La Chaîne des Côtes,388.2,77.6,[0.0],[11.0],6,4,"[[0, 9, 3], [9, 17, 0], [17, 25, 3], [25, 28, ...","[[0, 29, 2]]","[[0, 29, 0]]","[[0, 29, -1]]"
2,168474,Boucles du 13 - Saintes Maries,NaN,NaN,[0.0],[0.0],1,0,[],[],[],[]
3,168475,Les boucles du 13 - Autour de Silvacane,4528.4,905.7,[23.2],[12.2],2,0,"[[0, 56, 3]]","[[0, 56, 2]]","[[0, 56, 0]]","[[0, 56, 0]]"
4,168476,D’Arles à l’étang du Vaccarès,45.9,9.2,[1.0],[0.0],2,0,"[[0, 3, 3]]","[[0, 3, 2]]","[[0, 3, 0]]","[[0, 3, 0]]"
...,...,...,...,...,...,...,...,...,...,...,...,...
518,19816737,"Vallées, châtaignes et volcans",6426.4,1285.3,[387.3],[180.3],3,0,"[[0, 1, 0], [1, 270, 3]]","[[0, 3, 3], [3, 270, 2]]","[[0, 270, 0]]","[[0, 12, 3], [12, 27, -1], [27, 28, -5], [28, ..."
519,19824619,Uzès à Saint-Quentin-la-Poterie,3509.2,720.4,[9.0],[43.0],10,7,"[[0, 35, 3], [35, 36, 7], [36, 57, 3], [57, 66...","[[0, 15, 4], [15, 33, 3], [33, 35, 5], [35, 57...","[[0, 81, 0]]","[[0, 21, -1], [21, 81, 0]]"
520,19849376,V65,NaN,NaN,[0.0],[0.0],1,0,[],[],[],[]
521,19849377,V65,1213.6,285.3,[27.0],[24.0],10,6,"[[0, 4, 3], [4, 43, 0], [43, 62, 3]]","[[0, 4, 2], [4, 7, 4], [7, 14, 5], [14, 22, 3]...","[[0, 62, 0]]","[[0, 62, 1]]"


In [5]:
from google.cloud import storage
import os

def upload_json_folder_to_gcs(local_folder: str, bucket_name: str, destination_folder: str):
    """
    Uploads all JSON files from a local folder to a folder in a Google Cloud Storage bucket.

    Args:
        local_folder (str): Path to the local folder containing JSON files.
        bucket_name (str): Name of the Google Cloud Storage bucket.
        destination_folder (str): Folder path inside the bucket (prefix).
                                  Example: "data/json-files"

    Returns:
        None
    """
    # Initialize client
    client = storage.Client(project="cyclemore")
    bucket = client.bucket(bucket_name)

    # Ensure folder path ends correctly
    destination_folder = destination_folder.strip("/")

    # Loop over all files
    for filename in os.listdir(local_folder):
        if filename.endswith(".json"):
            local_path = os.path.join(local_folder, filename)
            blob_path = f"{destination_folder}/{filename}"  # Path in bucket

            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_path)

            print(f"Uploaded: {local_path} → gs://{bucket_name}/{blob_path}")


In [7]:
upload_json_folder_to_gcs(
    local_folder="/Users/eugeneleach/code/Eugle3/cycle_more/raw_ors_responses_Provence",
    bucket_name="cycle_more_bucket",
    destination_folder="raw_ors_data/provence"
)

Uploaded: /Users/eugeneleach/code/Eugle3/cycle_more/raw_ors_responses_Provence/route_9730960.json → gs://cycle_more_bucket/raw_ors_data/provence/route_9730960.json
Uploaded: /Users/eugeneleach/code/Eugle3/cycle_more/raw_ors_responses_Provence/route_10579284.json → gs://cycle_more_bucket/raw_ors_data/provence/route_10579284.json
Uploaded: /Users/eugeneleach/code/Eugle3/cycle_more/raw_ors_responses_Provence/route_17751305.json → gs://cycle_more_bucket/raw_ors_data/provence/route_17751305.json
Uploaded: /Users/eugeneleach/code/Eugle3/cycle_more/raw_ors_responses_Provence/route_10168097.json → gs://cycle_more_bucket/raw_ors_data/provence/route_10168097.json
Uploaded: /Users/eugeneleach/code/Eugle3/cycle_more/raw_ors_responses_Provence/route_13885176.json → gs://cycle_more_bucket/raw_ors_data/provence/route_13885176.json
Uploaded: /Users/eugeneleach/code/Eugle3/cycle_more/raw_ors_responses_Provence/route_196615.json → gs://cycle_more_bucket/raw_ors_data/provence/route_196615.json
Uploaded: 

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('belgium_turbo.csv')

In [4]:
df = df.head(2000)

In [5]:
df

,id,name,start_lon,start_lat,end_lon,end_lat,num_points
0,3770,Brialmontroute,4.467052,51.217621,4.352736,51.166423,2076
1,4086,Unnamed route,4.452915,51.176121,4.440786,51.189201,66
2,4166,Unnamed route,4.462921,51.215613,4.435500,51.194407,256
3,4167,Unnamed route,4.470557,51.217474,4.498876,51.243649,346
4,4207,Unnamed route,4.435345,51.194571,4.404426,51.187413,192
...,...,...,...,...,...,...,...
1995,192666,Unnamed route,5.804207,50.808150,5.809477,50.790368,221
1996,192668,Unnamed route,5.768687,50.796225,5.766045,50.808316,245
1997,192670,Unnamed route,5.765028,50.795865,5.772166,50.789989,142
1998,192672,Unnamed route,5.737332,50.810978,5.732519,50.796160,162
